In [1]:
import math
import statistics
import numpy
import csv
import scipy as sp
from scipy import interpolate
from scipy.stats import poisson
from scipy.stats import norm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns
from scipy.stats import kendalltau
import matplotlib
from matplotlib.ticker import FuncFormatter, MultipleLocator
import matplotlib.cm as cm
from matplotlib.widgets import Slider
from matplotlib.ticker import FuncFormatter
import matplotlib.ticker as ticker
import plotly.plotly as py
import cufflinks as cf
import plotly.graph_objs as go
py.sign_in('PythonAPI', 'ubpiol2cve')

/home/gaba/anaconda3/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [99]:
global_users = pd.read_csv('../../fb2015/users.csv', header=None)#.sample(frac=0.1)
global_users.columns = ['userId', 'registerCountry', 'signupTime']
global_conv = pd.read_csv('../../fb2015/conversions.csv', header=None)#.sample(frac=0.1)
global_conv.columns = ['userId', 'itemId', 'price', 'quantity', 'timestamp']
global_items = pd.read_csv('../../fb2015/items.csv', header=None)#.sample(frac=0.1)
global_items.columns = ['itemId', 'style', 'personality', 'color', 'theme', 'price', 'category']
global_users_ads = pd.read_csv('../../fb2015/users_ads.csv', header=None)#.sample(frac=0.1)
global_users_ads.columns = ['userId', 'utmSource', 'utmCampaign', 'utmMedium', 'utmTerm', 'utmContent']

Obtains data from files.

In [3]:
def drop_nulls(table, columns):
    table = table.dropna(subset=columns)
    for col in columns:
        table = table[~table[col].isin(['None'])]
    return table

Drops NaN, None values from given data.

In [ ]:
def number_of_new_users_per_month():
    users = drop_nulls(global_users, ['signupTime', 'userId'])

    users['month'] = pd.DatetimeIndex(users['signupTime']).month
    users['year'] = pd.DatetimeIndex(users['signupTime']).year

    grouped = users.groupby(['year', 'month'])['userId'].count()
    sortedd = grouped.sort_index()

    ax = sortedd.plot(kind='bar')

    ax.set_xlabel('Month')
    ax.set_ylabel('Number of new users')
    ax.set_title("Number of new users per month.", fontsize=18)

    plt.xticks(rotation=70)
    plt.tight_layout()
    plt.show()

Displays number of newly registered users per month.

In [ ]:
def number_of_conversions_per_month():
    conv = drop_nulls(global_conv, ['timestamp'])

    conv['month'] = pd.DatetimeIndex(conv['timestamp']).month
    conv['year'] = pd.DatetimeIndex(conv['timestamp']).year

    grouped = conv.groupby(['year', 'month']).count()
    sortedd = grouped.sort_index()

    ax = sortedd.plot(legend=None)
    ax.set_xlabel('Month')
    ax.set_ylabel('Number of conversions')
    ax.set_title("Number of conversions per month.", fontsize=18)

    plt.xticks(rotation=70)
    plt.tight_layout()

    plt.show()

Displays number of conversions per month.

In [ ]:
def number_of_items_purchased_per_month():
    conv = drop_nulls(global_conv, ['timestamp', 'quantity'])

    conv['month'] = pd.DatetimeIndex(conv['timestamp']).month
    conv['year'] = pd.DatetimeIndex(conv['timestamp']).year

    grouped = conv.groupby(['year', 'month'])['quantity'].sum()
    sortedd = grouped.sort_index()

    ax = sortedd.plot(kind='bar')
    ax.set_xlabel('Month')
    ax.set_ylabel('Number of items')
    ax.set_title("Number of items purchased per month.", fontsize=18)

    plt.xticks(rotation=70)
    plt.tight_layout()

    plt.show()

Displays number of items purchased per month.

In [ ]:
def revenue_per_month():
    conv = drop_nulls(global_conv, ['timestamp', 'price'])

    conv['month'] = pd.DatetimeIndex(conv['timestamp']).month
    conv['year'] = pd.DatetimeIndex(conv['timestamp']).year
    grouped = conv.groupby(['year', 'month'])['price'].sum()
    sortedd = grouped.sort_index()

    ax = sortedd.plot() 
    ax.set_xlabel('Month')
    ax.set_ylabel('Revenue')
    ax.set_title("Revenue per month.", fontsize=18)

    plt.xticks(rotation=70)
    plt.tight_layout()

    plt.show()

Displays income per month.

In [ ]:
def number_of_purchases_per_user_in_the_first_week_after_registration():
    #does not include users without purchase in the first week after registration
    users = drop_nulls(global_users, ['userId', 'signupTime'])
    conv = drop_nulls(global_conv, ['userId', 'quantity', 'timestamp'])

    users['signupTime'] =  pd.DatetimeIndex(users['signupTime'])
    users['week_after'] = pd.DatetimeIndex(users['signupTime'])+pd.Timedelta(days=7)
    conv['timestamp'] = pd.DatetimeIndex(conv['timestamp'])

    joined = pd.merge(users, conv, left_on='userId', right_on='userId', how='inner')
    grouped = joined[joined['timestamp'] <= joined['week_after']].groupby('userId')['quantity'].sum()

    ax = grouped.plot(kind='bar')
    ax.set_xlabel('User id')
    ax.set_ylabel('Quantity')
    ax.set_title("Number of items purchased in the first week after registration, grouped by userId.", fontsize=18)

    plt.xticks(rotation=70)
    plt.tight_layout()

    plt.show()

Displays number of items purchased during the first week after signing in.

In [ ]:
def number_of_items_purchased_from_particular_category_grouped_by_country(category):
    category = 2346301904
    conv = drop_nulls(global_conv, ['quantity'])
    items = drop_nulls(global_items, ['category'])
    users = drop_nulls(global_users, ['registerCountry'])

    joined_items_conv = pd.merge(items, conv, left_on='itemId', right_on='itemId', how='inner')
    joined = pd.merge(joined_items_conv, users, left_on='userId', right_on='userId', how='inner')

    filtered = joined[joined['category']==category]
    grouped = filtered.groupby('registerCountry')['quantity'].sum()

    ax = grouped.plot(kind='bar')
    ax.set_xlabel('Category')
    ax.set_ylabel('Quantity')
    ax.set_title("Number of items from "+str(category)+" category, grouped by country.", fontsize=18)

    plt.xticks(rotation=70)
    plt.tight_layout()
    plt.show()

Displays number of items from particular category purchased by people from all countries.

In [ ]:
def number_of_items_purchased_in_particular_country_grouped_by_category(country):
    country = 'United States'
    conv = drop_nulls(global_conv, ['quantity'])
    items = drop_nulls(global_items, ['category'])
    users = drop_nulls(global_users, ['registerCountry'])

    joined_items_conv = pd.merge(items, conv, left_on='itemId', right_on='itemId', how='inner')
    joined = pd.merge(joined_items_conv, users, left_on='userId', right_on='userId', how='inner')

    filtered = joined[joined['registerCountry']==country]
    grouped = filtered.groupby('category')['quantity'].sum()

    ax = grouped.plot(kind='bar')
    ax.set_xlabel('Category')
    ax.set_ylabel('Quantity')
    ax.set_title("Number of items purchased in " + country + ", grouped by category.", fontsize=18)

    plt.xticks(rotation=70)
    plt.tight_layout()
    plt.show()

Displays number of items from all cattegories purchased in particular country.

In [15]:
def number_of_purchased_items_grouped_by_categories_in_all_countries():
    conv = drop_nulls(global_conv, ['quantity'])
    items = drop_nulls(global_items, ['category'])
    users = drop_nulls(global_users, ['registerCountry'])

    joined_items_conv = pd.merge(items, conv, left_on='itemId', right_on='itemId', how='inner')
    joined = pd.merge(joined_items_conv, users, left_on='userId', right_on='userId', how='inner')

    y_labels = []
    x_labels = []
    y_ticks = []
    x_ticks = []

    for i, row in enumerate(joined['registerCountry'].unique()):
        y_labels.append(row)
        y_ticks.append(i)

    for i, row in enumerate(joined['category'].unique()):
        x_labels.append(row)
        x_ticks.append(i)

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)

    for j, country in enumerate(y_labels):
        r = np.random.rand(1)
        color = cm.rainbow(np.linspace(r, r+1, 1))       
        for i, category in enumerate(x_labels):
            countryFiltered = joined[joined['registerCountry']==country]
            filtered = countryFiltered[countryFiltered['category']==category]
            area=filtered['quantity'].sum()
            ax.scatter(x=i, y=j, s=area, c=color, alpha=0.7)  

    ax.set(xticks=range(len(x_labels)), xticklabels=x_labels,
       yticks=range(len(y_labels)), yticklabels=y_labels)

    ax.set_xlabel('Category')
    ax.set_ylabel('Country')
    ax.set_title('Sum of purchased items, grouped by country and category.', fontsize=18)

    plt.tick_params(axis='x')
    plt.xticks(rotation=90)
    plt.tight_layout()

    plt.show()

Displays general view on sum of purchased items grouped by country and category.

In [4]:
def number_of_purchased_items_after_seeing_campaigns_grouped_by_categories():
    conv = drop_nulls(global_conv, ['timestamp'])
    users_ads = drop_nulls(global_users_ads, ['utmCampaign'])
    users = drop_nulls(global_users, ['signupTime'])
    items = drop_nulls(global_items, ['category'])

    users['week_after'] = pd.DatetimeIndex(users['signupTime'])+pd.Timedelta(days=7)
    conv['timestamp'] = pd.DatetimeIndex(conv['timestamp'])

    joined = pd.merge(users, users_ads, left_on='userId', right_on='userId', how='inner')
    joined = pd.merge(joined, conv, left_on='userId', right_on='userId', how='inner')
    joined = pd.merge(joined, items, left_on='itemId', right_on='itemId', how='inner')

    filtered = joined[joined['timestamp'] <= joined['week_after']]

    y_labels = []
    x_labels = []
    y_ticks = []
    x_ticks = []
    x = []
    y = []

    for i, row in enumerate(filtered['utmCampaign'].unique()):
        y_labels.append(row)
        y_ticks.append(i)
        y.append(i)

    for i, row in enumerate(filtered['category'].unique()):
        x_labels.append(row)
        x_ticks.append(i)
        x.append(i)

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)

    for j, utmCampaign in enumerate(y_labels):
        r = np.random.rand(1)
        color = cm.rainbow(np.linspace(r, r+1, 1))       
        for i, category in enumerate(x_labels):
            utmCampaignFiltered = filtered[filtered['utmCampaign']==utmCampaign]
            categoryFiltered = utmCampaignFiltered[utmCampaignFiltered['category']==category]
            area=categoryFiltered['quantity'].sum()
            ax.scatter(x=i, y=j, s=area, c=color, alpha=0.7)  

    ax.set(xticks=range(len(x_labels)), xticklabels=x_labels,
       yticks=range(len(y_labels)), yticklabels=y_labels)

    ax.set_xlabel('Category')
    ax.set_ylabel('utmCampaign')
    ax.set_title('Sum of purchased items in the first week after signing in, grouped by campaigns and category.', fontsize=18)

    plt.tick_params(axis='x', labelsize=8)
    plt.xticks(rotation=70)
    plt.tight_layout()

    plt.show()

/home/ducki13/anaconda3/lib/python3.5/site-packages/numpy/lib/arraysetops.py:379: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/home/ducki13/anaconda3/lib/python3.5/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


Displays general view on sum of purchased items in the first week after signing in, connected with campaigns being seen, grouped by category and campaign.

In [2]:
def average_number_of_purchased_items_during_the_first_month_after_signing_in():
    conv = drop_nulls(global_conv, ['timestamp', 'quantity'])
    users = drop_nulls(global_users, ['signupTime', 'userId'])

    users['signupTime'] = pd.DatetimeIndex(users['signupTime'])
    conv['timestamp'] = pd.DatetimeIndex(conv['timestamp'])

    joined = pd.merge(users, conv, left_on='userId', right_on='userId', how='inner')
    joined['purchase_day'] = pd.to_timedelta(joined['timestamp'])-pd.to_timedelta(joined['signupTime'])

    filtered = joined[joined['purchase_day']>=pd.Timedelta(days=0)]
    filtered = filtered[filtered['purchase_day']<pd.Timedelta(days=30)]
    filtered['purchase_day'] = pd.DatetimeIndex(filtered['purchase_day']).day

    aver = filtered.groupby(['purchase_day', 'userId'])['quantity'].sum()

    summ = [0 for x in range(0,31)]
    countt = [0 for x in range(0,31)]
    average = [0 for x in range(0,31)]

    for x in aver.iteritems():
        day = x[0][0]
        quantitySum = x[1]
        summ[day] += quantitySum
        countt[day] += 1

    for y in range(0, 31):
        if countt[y] != 0:
            average[y] = summ[y]/countt[y]

    ax = sns.barplot(y=average, x=[i for i in range(0, 31)])
    ax.set_xlabel('Day after signing in')
    ax.set_ylabel('Average quantity')
    ax.set_title('Average sum of purchases (quantities) during first month after signing in.', fontsize=18)

    plt.tick_params(axis='x', labelsize=8)
    plt.tight_layout()

    plt.show()

Displays average sum of purchases during first month after signing in.

In [6]:
def distribution_number_of_purchases_per_user_in_the_first_week_after_registration():
    users = drop_nulls(global_users, ['signupTime'])
    conv = drop_nulls(global_conv, ['timestamp', 'quantity'])

    users['signupTime'] =  pd.DatetimeIndex(users['signupTime'])
    users['week_after'] = pd.DatetimeIndex(users['signupTime'])+pd.Timedelta(days=7)
    conv['timestamp'] = pd.DatetimeIndex(conv['timestamp'])

    joined = pd.merge(users, conv, left_on='userId', right_on='userId', how='inner')
    filtered = joined[joined['timestamp'] <= joined['week_after']]
    grouped = filtered.groupby('userId')['quantity'].sum()

    d = dict()
    for x in grouped:
        if x not in d.keys():
            d[x] = 1
        else:
            d[x] += 1

    d[0] = len(joined[~joined['userId'].isin(filtered['userId'].unique())].groupby('userId'))

    layout = go.Layout(
        title='Distribution of sum of purchases per user during first week after registration',
        xaxis=dict(
            title='Number of purchased products',
            titlefont=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        ),
        yaxis=dict(
            title='Number of users',
            titlefont=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        )
    )
    data = [go.Bar(x=list(d.keys()), y=list(d.values()))]
    fig = go.Figure(data=data, layout=layout)
    py.plot(fig, filename='pandas-bar-chart')

'https://plot.ly/~PythonAPI/1438'

Shows distribution of how many items users bought during first week after sign up.

In [17]:
def probability_of_purchase_during_the_first_month_after_registration():
    conv = drop_nulls(global_conv, ['timestamp', 'quantity'])
    users = drop_nulls(global_users, ['signupTime'])

    users['signupTime'] = pd.DatetimeIndex(users['signupTime'])
    conv['timestamp'] = pd.DatetimeIndex(conv['timestamp'])

    joined = pd.merge(users, conv, left_on='userId', right_on='userId', how='inner')
    joined['purchase_day'] = pd.to_timedelta(joined['timestamp'])-pd.to_timedelta(joined['signupTime'])

    nr_of_users = len(joined.groupby(['userId']))

    filtered = joined[joined['purchase_day']>=pd.Timedelta(days=0)]
    filtered = filtered[filtered['purchase_day']<pd.Timedelta(days=30)]
    filtered['purchase_day'] = pd.DatetimeIndex(filtered['purchase_day']).day

    purch = filtered.groupby(['purchase_day'])['userId'].count()
    probabil = [0 for x in range(0,31)]

    for x in purch.iteritems():
        day = x[0]
        probabil[day] = x[1]/nr_of_users

    ax = sns.barplot(y=probabil, x=[i for i in range(0, 31)])
    ax.set_xlabel('Day after registration')
    ax.set_ylabel('Probability (sum of purchases/number of users who have at least one purchase)')
    ax.set_title('Probability of purchase during the first month after registration', fontsize=18)

    plt.tick_params(axis='x', labelsize=8)
    plt.tight_layout()

    plt.show()

/home/gaba/anaconda3/lib/python3.5/site-packages/numpy/lib/arraysetops.py:379: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison

/home/gaba/anaconda3/lib/python3.5/site-packages/matplotlib/__init__.py:892: UserWarning:

axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.



Shows how probably is user's purchase during the first month after sign up.

In [4]:
def poiss(l, r):
    return (l**r * math.e**(-l))/math.factorial(r)

In [31]:
def poisson_distribution_weekly_user_count_of_purchases():
    conv = drop_nulls(global_conv, ['timestamp'])

    conv['week'] = pd.DatetimeIndex(conv['timestamp']).week
    conv['year'] = pd.DatetimeIndex(conv['timestamp']).year

    grouped = conv.groupby(['year', 'week'])['quantity'].sum()
    aver = math.floor(grouped.mean())

    val = []
    [val.append(0) for x in range(0, max(grouped.values) + 1)]

    for x in grouped.values:
        val[x] += 1

    mu = math.floor(grouped.mean())

    counts = poisson.rvs(mu, size=100)
    bins = range(0, max(counts)+2)

    plt.hist(counts, bins=bins, align="left", histtype="step", color="black")

    x = range(0,max(counts)*2)
    prob = poisson.pmf(x, mu)*100 

    plt.plot(x, prob)
    
    ax.set_xlabel('Day after registration')

    plt.show()

In [106]:
def to_percent(y, position):
    s = str(y)
    if matplotlib.rcParams['text.usetex'] is True:
        return s + r'$\%$'
    else:
        return s + '%'

In [19]:
#def inverse_gamma_distribution(alpha, beta, gamma, mi):
from scipy.stats import invgamma
numargs = invgamma.numargs

y = [i for i in range(0, 5)]

mle = np.mean(y)
rv = invgamma(mle)
x = np.linspace(0, np.minimum(rv.dist.b, 5))

h = plt.plot(x, rv.pdf(x))
plt.show()

In [108]:
def poisson_distribution_weekly_user_count_of_purchases_2():
    conv = drop_nulls(global_conv, ['timestamp'])

    conv['week'] = pd.DatetimeIndex(conv['timestamp']).week
    conv['year'] = pd.DatetimeIndex(conv['timestamp']).year

    grouped = conv.groupby(['year', 'week'])['quantity'].sum()
    aver = math.floor(grouped.mean())

    purch_sum = sum(grouped.values)
    mu = math.floor(grouped.mean())

    counts = poisson.rvs(mu, size=100)
    bins = range(0, max(counts)+2)
    bins = bins[0::10]

    #Histogram
    weights=100*np.ones_like(grouped.values)/len(grouped.values)
    ax = plt.hist(grouped.values, weights=weights, bins=bins, align="left", histtype="step", color="black", label='Histogram of the data') 

    #Poisson
    x = range(0,2*max(counts))
    prob = poisson.pmf(x, mu)*100
    plt.plot(x, prob, label='Poisson Distribution')

    plt.tick_params(axis='x', labelsize=8)
    plt.legend(bbox_to_anchor=(0.7, 0.9), loc=2, borderaxespad=0.)
    plt.title('Poisson Distribution of weekly users\' count of purchases', fontsize=18)

    ax = plt.gca()
    ax.set_xlabel('Quantity')
    ax.set_ylabel('Percent')

    plt.tight_layout()
    plt.show()

In [115]:
#def number_of_active_users_per_month():
users = global_users
conv = drop_nulls(global_conv, ['timestamp'])

conv['month'] = pd.DatetimeIndex(conv['timestamp']).month
conv['year'] = pd.DatetimeIndex(conv['timestamp']).year

joined = pd.merge(users, conv, left_on='userId', right_on='userId', how='inner')
grouped = joined.groupby(['year', 'month'])['userId'].unique().count()
#sortedd = grouped.sort_index()

ax = sortedd.plot(kind='bar')

ax.set_xlabel('Month')
ax.set_ylabel('Number of active users')
ax.set_title("Number of active users per month.", fontsize=18)

plt.xticks(rotation=70)
plt.tight_layout()
plt.show()

Number of users who have at least one purchase in particular month.

In [ ]:
#TODO
#def percent_of_active_users_per_month():
users = global_users
conv = drop_nulls(global_conv, ['timestamp'])

conv['month'] = pd.DatetimeIndex(conv['timestamp']).month
conv['year'] = pd.DatetimeIndex(conv['timestamp']).year

joined = pd.merge(users, conv, left_on='userId', right_on='userId', how='inner')
grouped = joined.groupby(['year', 'month'])['userId'].unique().count()
#sortedd = grouped.sort_index()

ax = sortedd.plot(kind='bar')

ax.set_xlabel('Month')
ax.set_ylabel('Percent of active users')
ax.set_title("Percent of active users per month.", fontsize=18)

plt.xticks(rotation=70)
plt.tight_layout()
plt.show()

In [109]:
#TODO
#def variance_sum_of_revenue_for_items_purchased_during_first_month_after_signup():
users = drop_nulls(global_users, ['userId'])
conv = drop_nulls(global_conv, ['timestamp'])

conv['conv_month'] = pd.DatetimeIndex(conv['timestamp']).month
conv['conv_year'] = pd.DatetimeIndex(conv['timestamp']).year

users['signup_month'] = pd.DatetimeIndex(users['signupTime']).month
users['signup_year'] = pd.DatetimeIndex(users['signupTime']).year

joined = pd.merge(users, conv, left_on='userId', right_on='userId', how='inner')
grouped = joined.groupby(['conv_year', 'conv_month', 'userId'])['price'].sum()
grouped = grouped.sort_index()
signup_grouped = users.groupby(['signup_year', 'signup_month'])['userId'].count()
signup_grouped = signup_grouped.sort_index()
for i, x in enumerate(signup_grouped):
    if i != 0:
        signup_grouped[i] += signup_grouped[i-1]

signed = []
for i in range(0, 3):
    signed.append([])
    for j in range(0, 13):
        signed[i].append(0)

print("len="+str(len(signup_grouped)))
print(len(signed[0]))
for i, year_id in enumerate(signup_grouped.index.labels[0]):
    month_id = signup_grouped.index.labels[1][i]
    year = signup_grouped.index.levels[0][year_id]
    month = signup_grouped.index.levels[1][month_id]
    print("y:"+str(year-2013)+" m:"+str(month)+" i:"+str(i))
    signed[year-2013][month] = signup_grouped[i]
    
res = []
for i in range(0, 3):
    res.append([])
    for j in range(0, 13):
        res[i].append(0)

#count variance for each month
m = -1
y = -1
data = []
for i, year_id in enumerate(grouped.index.labels[0]):
    month_id = grouped.index.labels[1][i]
    year = grouped.index.levels[0][year_id]
    month = grouped.index.levels[1][month_id]
    if(i != 0 and (y != year or m != month)) or (i == len(grouped.index.labels[0])-1):
        if i == len(grouped.index.labels[0])-1:
            data.append(grouped[i])
        if signed[y-2013][m] > 0:
            data.extend([0] * (signed[y-2013][m] - len(data)))
            res[y-2013][m] = statistics.variance(data)
        data = []

    data.append(grouped[i])
    y = year
    m = month

variance = []
labels = []                                                 
for i in range(0, 3):
    for j in range(0, 13):
        variance.append(res[i][j])
        labels.append(str(j)+" / "+str(i+2013))
                                                 
ax = sns.barplot(y=variance, x=labels)
ax.set_xlabel('Year and month')
ax.set_ylabel('Variance')
ax.set_title('Variance: sum of prices of purchases for each user in each month after sign up', fontsize=18)

plt.tick_params(axis='x', labelsize=10)
plt.xticks(rotation=70)
plt.tight_layout()

plt.show()

len=33
13
y:0 m:1 i:0
y:0 m:2 i:1
y:0 m:3 i:2
y:0 m:4 i:3
y:0 m:5 i:4
y:0 m:6 i:5
y:0 m:7 i:6
y:0 m:8 i:7
y:0 m:9 i:8
y:0 m:10 i:9
y:0 m:11 i:10
y:0 m:12 i:11
y:1 m:1 i:12
y:1 m:2 i:13
y:1 m:3 i:14
y:1 m:4 i:15
y:1 m:5 i:16
y:1 m:6 i:17
y:1 m:7 i:18
y:1 m:8 i:19
y:1 m:9 i:20
y:1 m:10 i:21
y:1 m:11 i:22
y:1 m:12 i:23
y:2 m:1 i:24
y:2 m:2 i:25
y:2 m:3 i:26
y:2 m:4 i:27
y:2 m:5 i:28
y:2 m:6 i:29
y:2 m:7 i:30
y:2 m:8 i:31
y:2 m:9 i:32


/home/gaba/anaconda3/lib/python3.5/site-packages/matplotlib/__init__.py:892: UserWarning:

axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.



In [10]:
def stanDev(values):
    length = len(values)
    m = statistics.mean(values)
    total_sum = 0
    for i in range(length):
        total_sum += (values[i]-m)**2
 
    return math.sqrt(total_sum/length)

In [11]:
def normDistr(x, m, stDev):
    return (math.e**((-((x-m)**2))/(2*stDev)))/(math.sqrt(stDev)*math.sqrt(2*math.pi))

In [28]:
#def normal_distribution_weekly_revenue:
conv = drop_nulls(global_conv, ['timestamp'])

conv['year'] = pd.DatetimeIndex(conv['timestamp']).year
conv['week'] = pd.DatetimeIndex(conv['timestamp']).week

grouped = conv.groupby(['year', 'week'])['price'].sum()

val = []
idx = []
_mean = statistics.mean(grouped.values)
_stDev = stanDev(grouped.values)

grouped.values.sort()
#for x in grouped.values:
norm(loc=_mean, scale=_stDev)
print(norm.rvs(size=100))
    #a = normDistr(x, _mean, _stDev)
    #val.append(a)
#    idx.append(x)
'''
layout = go.Layout(
    title='Normal distribution of weekly revenue',
    xaxis=dict(
        title='Weekly revenue',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Value',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
data = [go.Bar(x=idx, y=val)]
fig = go.Figure(data=data, layout=layout)
py.plot(fig, filename='pandas-bar-chart')'''

[ 0.73764279  0.93108089  1.48549938  0.7890756  -0.74842187 -0.30902568
  0.90746309  0.24658256  1.4595704  -1.69814857  1.30022919  0.27659568
  0.00980337  0.70006099 -0.71187976  2.06677325 -2.2148354   0.03674409
 -0.59199395  0.17232559 -0.78274668  0.61294044  0.29687402 -0.188394
 -0.0514779   0.01749238 -1.66881192 -0.16706183 -1.29904777 -0.66001723
  1.85322503 -0.3822971   1.27169724 -0.3773073  -0.48601619 -0.39422666
 -2.18095573  1.15202879 -0.79130947 -0.76726924 -1.36314302  1.0285783
  0.24828003 -1.01015527 -0.17483492 -0.36619198 -0.7937611   0.57816292
 -0.98268645  0.03410332 -1.34399696 -0.01258483 -0.6406245   0.77057919
 -0.36142031  0.28804083  0.57164363 -0.56586091 -0.14879263 -2.56639428
  1.19969603  0.66398231  1.52907217 -0.40550384  0.01526271 -0.58821857
  0.15991023 -0.17807189  2.10622713 -0.01647444  0.32623958  0.28417721
  0.48121597  0.14437613  1.11337862  0.82636676 -1.96879981 -0.40812348
  0.18661677 -0.5486418  -2.97274    -0.98322878 -0.56

"\nlayout = go.Layout(\n    title='Normal distribution of weekly revenue',\n    xaxis=dict(\n        title='Weekly revenue',\n        titlefont=dict(\n            family='Courier New, monospace',\n            size=18,\n            color='#7f7f7f'\n        )\n    ),\n    yaxis=dict(\n        title='Value',\n        titlefont=dict(\n            family='Courier New, monospace',\n            size=18,\n            color='#7f7f7f'\n        )\n    )\n)\ndata = [go.Bar(x=idx, y=val)]\nfig = go.Figure(data=data, layout=layout)\npy.plot(fig, filename='pandas-bar-chart')"